In [0]:
#INITIALIZATION
import pyspark.sql.functions as F
from pyspark.sql.types import StringType
from pyspark.sql.functions import trim, col

In [0]:
#READ BRONZE TABLE
df = spark.table("workspace.bronze.erp_cust_az12")

In [0]:
#TRIMMING
for field in df.schema.fields:
    if isinstance(field.dataType, StringType):
        df = df.withColumn(field.name, trim(col(field.name)))

In [0]:
#Customer ID Cleanup

df = df.withColumn(
    "cid",
    F.when(col("cid").startswith("NAS"),
           F.substring(col("cid"), 4, F.length(col("cid"))))
     .otherwise(col("cid"))
)

In [0]:
#Birthdate Validation

df = df.withColumn(
    "bdate",
    F.when(col("bdate") > F.current_date(), None)
     .otherwise(col("bdate"))
)

In [0]:
#Gender Normalization

df = df.withColumn(
    "gen",
    F.when(F.upper(col("gen")).isin("F", "FEMALE"), "Female")
     .when(F.upper(col("gen")).isin("M", "MALE"), "Male")
     .otherwise("n/a")
)

In [0]:
#Renaming Columns

RENAME_MAP = {
    "cid": "customer_number",
    "bdate": "birth_date",
    "gen": "gender"
}
for old_name, new_name in RENAME_MAP.items():
    df = df.withColumnRenamed(old_name, new_name)

In [0]:
#Sanity checks of dataframe

df.limit(10).display()

In [0]:
#Writing Silver Table

df.write.mode("overwrite").format("delta").saveAsTable("workspace.silver.erp_customers")

In [0]:
%sql
---Sanity checks of silver table--
SELECT * FROM workspace.silver.erp_customers LIMIT 10